In [14]:
import cv2
import numpy as np
import yaml
import os

# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Initialize the video capture (0 for default camera)
cap = cv2.VideoCapture(1)

# Load calibration data which was previously done with ArUco markers
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Get the optimal camera matrix
ret, img = cap.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # GrayScale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Initialize start and end points
    start_point = None
    end_point = None

    if ids is not None:
        # Detect start (6) and end (5) markers
        for i, marker_id in enumerate(ids.flatten()):
            marker_center = np.mean(corners[i][0], axis=0)  # Center of the marker

            if marker_id == 6:  # START
                start_point = marker_center
                cv2.circle(frame, tuple(start_point.astype(int)), 5, (0, 255, 0), -1)  # Draw green circle

            elif marker_id == 5:  # END
                end_point = marker_center
                cv2.circle(frame, tuple(end_point.astype(int)), 5, (0, 0, 255), -1)  # Draw red circle

        # Check if all four required corner markers are detected
        required_ids = {1, 2, 3, 4}
        marker_positions = {marker_id: corners[i][0] for i, marker_id in enumerate(ids.flatten())}
        if required_ids.issubset(marker_positions.keys()):
            rect_corners = np.array([
                marker_positions[1][0],  # Top-left corner
                marker_positions[2][1],  # Top-right corner
                marker_positions[3][2],  # Bottom-right corner
                marker_positions[4][3],  # Bottom-left corner
            ], dtype=np.float32)

            # Ensure rect_corners has the correct shape
            if rect_corners.shape == (4, 2):
                # Draw rectangle around the detected markers
                cv2.polylines(frame, [np.int32(rect_corners)], isClosed=True, color=(0, 0, 255), thickness=3)

                # Define destination points for the flattened rectangle (6:8 aspect ratio)
                width, height = 600, 800  # Desired output size with 6:8 aspect ratio
                dst_corners = np.array([
                    [0, 0],
                    [width - 1, 0],
                    [width - 1, height - 1],
                    [0, height - 1]
                ], dtype=np.float32)

                # Compute the perspective transform matrix
                matrix = cv2.getPerspectiveTransform(rect_corners, dst_corners)

                # Apply perspective warp to get the top-down view
                warped_image = cv2.warpPerspective(frame, matrix, (width, height))

                # Convert warped image to grayscale and threshold it to create a binary map
                gray_warped = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)
                _, binary_map = cv2.threshold(gray_warped, 100, 255, cv2.THRESH_BINARY_INV)

                padding_x = -10  # Horizontal padding
                padding_y = 10  # Vertical padding

                circle_padding = 100
                
                # Mask ArUco markers 5 and 6
                for i, marker_id in enumerate(ids.flatten()):
                    
                    if marker_id in {5, 6}:  # Mask START (6) and END (5)
                        # Transform the center of the marker using the perspective transform matrix
                        marker_center = np.mean(corners[i][0], axis=0)  # Center of the marker
                        warped_center = cv2.perspectiveTransform(np.array([[marker_center]], dtype=np.float32), matrix)[0][0]
                        
                        # Convert center coordinates to integers
                        warped_center_int = tuple(np.clip(warped_center.astype(int), 0, [binary_map.shape[1] - 1, binary_map.shape[0] - 1]))
                        
                        # Mask the circular region in the binary map
                        cv2.circle(binary_map, warped_center_int, circle_padding, 0, -1)  # Set to white (255)
                
                # Create a grid map for pathfinding
                grid_map = np.zeros_like(binary_map, dtype=int)

                # Set obstacle regions (binary_map == 255)
                grid_map[binary_map == 255] = 1

                # Set start and end points in the grid map
                if start_point is not None:
                    start_warped = cv2.perspectiveTransform(np.array([[start_point]], dtype=np.float32), matrix)[0][0]
                    start_coords = tuple(start_warped.astype(int))
                    grid_map[start_coords[1], start_coords[0]] = 3  # Start

                if end_point is not None:
                    end_warped = cv2.perspectiveTransform(np.array([[end_point]], dtype=np.float32), matrix)[0][0]
                    end_coords = tuple(end_warped.astype(int))
                    grid_map[end_coords[1], end_coords[0]] = 2  # End

                # Show warped image and binary map
                cv2.imshow('Warped Image', warped_image)
                color_map = np.zeros((*binary_map.shape, 3), dtype=np.uint8)
                color_map[binary_map == 0] = (255, 255, 255)
                color_map[binary_map == 255] = (0, 0, 0)
                color_map[grid_map == 2] = (0, 0, 255)  # End RED
                color_map[grid_map == 3] = (0, 255, 0)  # Start GREEN
                cv2.imshow('Binary Map', color_map)

                # Save maps to a folder
                current_dir = os.getcwd()
                folder_name = os.path.join(current_dir, "Maps")
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)
                np.savetxt(os.path.join(folder_name, "grid_map_6_8.txt"), grid_map, fmt='%d', delimiter=' ', comments='')

    # Show the original frame
    cv2.imshow('ArUco Marker Detection', frame)

    # Exit on 'q'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


AttributeError: module 'cv2.aruco' has no attribute 'detectMarkers'